In [1]:
%pip install --upgrade polars

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl

In [3]:
#not paarsing dates so I can first perform string operation on it
df_old = pl.read_csv("/home/anopsy/Portfolio/junior_roles/9june2023.csv", try_parse_dates=False)
df_new = pl.read_csv("/home/anopsy/Portfolio/junior_roles/2april2024.csv", try_parse_dates=False)


In [4]:
df_old = df_old.with_columns(sample_date = pl.lit("2023"))
df_new = df_new.with_columns(sample_date = pl.lit("2024"))

In [5]:
whole_df = df_old.vstack(df_new)

1. Compensation
   Compensation variable has a lot of issues. First of all it contains compensation in different currencies.
   I'll convert the local currency of the most numerous countries, which are India, Mexico, Canada, Poland. I'll skip European Countries with EURO as their currency, as the conversion rate is almost 1:1.
   The other issue I faced is the fact that the compensation can be expressed both as a monthly compensation for a full-time job. But it also contains values that look like an hourly wage. I'll assume that those are ineed hourly -reates and if the job is specified as full-time I'll multiply it by a full-time number of hours.
   The third problem is that some comp_est string are amalgamations of two strings- lower and higher range. I'll then split them and I'll compute the average.

a) amalgamated strings into two strings -> float/int -> average
There are 108 records that contain strings that contain lower and higher range of the compensation enstimation.


What did I do, I created a column with the length of the comp_est string and focused only on the strings that are longer than 9 chars. Amalgamated strings (eg 120000160000) have a specific structure so I extracted those using regex. I ended up with 108 records with lower and higher bound.

The logic is not perfect, I also generated 13 values, consisting of 3 extracted values, in 5 of them the first string is the lower bound and the two last striings combined will create the higher bound. In 4 other records there's an opposite problem, the first two string makes the lower bound while the two last strings can be combined into the higher bound. 4 other records doesn't make too much sense. I decided I wil therefore skip this step as creating a logic for cleaning only 8 records in total is in my opinion a suboptimal solution. But I could return to it as an exercise.


In [6]:
whole_df = whole_df.with_columns(len_comp = pl.col("comp_est").str.len_chars())

In [7]:
whole_df["len_comp"].value_counts(sort=True)

len_comp,count
u32,u32
null,47737
0,36939
9,5485
8,4642
4,1654
…,…
12,19
17,17
2,6


In [8]:
whole_df.filter(pl.col("len_comp")>9).select("comp_est")

comp_est
str
"""123649.825"""
"""2500000000.00"""
"""32299.999999999996"""
"""133293.495"""
"""130000160000.00"""
…
"""79770.76000000001"""
"""1450000.00"""
"""19800000.00"""


In [9]:
whole_df.filter(pl.col("len_comp")>10, pl.col("comp_est").str.contains("[123456789]+[0]+[123456789]+[0]+")).with_columns(est=pl.col("comp_est").str.extract_all("[123456789]+[0]+")).filter(pl.col("est").list.len()==2)

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,u32,list[str]
"""Fluid Test Engineer (Thermal)""","""Titusville, FL""","""Full-Time""",null,"""Latitude""",null,"""[Microsoft]""","""[]""","""[Government, Recruiting & Staf…","""Manager""","""130000160000.00""","""en""","""Titusville""","""United States""",null,"""2023-06-05 06:32:02""","""2023""",15,"[""130000"", ""160000""]"
"""Field Engineer I - Southern Co…",null,"""Full-Time""",null,"""Colorado Opportunities""","""Bachelors""","""[Microsoft]""","""[]""","""[]""",null,"""65000735000.00""","""en""","""Colorado Springs""","""United States""","""2023-05-26 15:13:27""","""2023-06-06 20:21:03""","""2023""",14,"[""65000"", ""735000""]"
"""Platform Security Engineer""","""Los Angeles, CA""","""Full-Time""",null,"""StubHub""",null,"""[C#, Java]""","""[Programming Languages, OSS]""","""[Security, Software]""",null,"""200000350000.00""","""en""","""Los Angeles""","""United States""","""2022-11-16 00:00:00""","""2023-05-24 04:46:37""","""2023""",15,"[""200000"", ""350000""]"
"""GRC Security Consultant (Gover…","""United States""","""Contract""",null,"""Braintrust""",null,"""[DFS, GDPR]""","""[Print on Demand, Compliance, …","""[Consulting, Compliance, Freel…","""Contract""","""230250000.00""","""en""",null,"""United States""","""2023-05-30 00:00:00""","""2023-05-31 23:10:51""","""2023""",12,"[""230"", ""250000""]"
"""Experienced Instrumentation & …","""Richmond The Standard Cafeteri…","""Full-Time""",null,"""0061 Chevron Products Company""","""Bachelors""","""[]""","""[]""","""[Business, Chemicals, Oil & Ga…","""Senior IC""","""112200221900.00""","""en""",null,"""United States""","""2023-06-02 00:00:00""","""2023-06-08 04:58:36""","""2023""",15,"[""112200"", ""221900""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Program Manager, Partnerships …","""United States""","""Full-Time""",null,"""Braintrust""","""Bachelors""","""[]""","""[]""","""[]""","""Manager""","""150000177000.00""","""en""",null,"""United States""","""2023-10-17 00:00:00""","""2024-03-26 12:28:35""","""2024""",15,"[""150000"", ""177000""]"
"""Full-Stack Java Lead""","""Houston, Texas, 77001 United S…","""Full-Time""",null,"""KP Recruiting Group""",null,"""[Terraform, Red Hat, Docker, B…","""[Enterprise Software, SaaS, OS…","""[Recruiting & Staffing, IT]""",null,"""130000140000.00""","""en""","""Houston""","""United States""","""2022-02-01 00:00:00""","""2024-03-25 19:14:06""","""2024""",15,"[""130000"", ""140000""]"
"""Mid-level Environmental Engine…","""Denver, Colorado, United State…","""Full-Time""",null,"""Integral Consulting""","""Masters""","""[]""","""[]""","""[Dispute Resolution, Consultin…",null,"""75000100000.00""","""en-us""","""Denver""","""United States""","""2023-03-24 00:00:00""","""2024-03-22 04:45:58""","""2024""",14,"[""75000"", ""100000""]"


In [10]:
whole_df = whole_df.with_columns(
    est=pl.when((pl.col("len_comp")>10) & (pl.col("comp_est").str.contains("[123456789]+[0]+[123456789]+[0]+")))
            .then(pl.col("comp_est").str.extract_all("[123456789]+[0]+"))
            .otherwise(pl.concat_list("comp_est", "comp_est")))


In [11]:
whole_df = whole_df.with_columns(est_float = pl.col("est").cast(pl.List(pl.Float64)))


In [12]:
whole_df = whole_df.with_columns(compensation = pl.col("est_float").list.mean())

In [13]:
whole_df.filter(pl.col("compensation")>0)

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,u32,list[str],list[f64],f64
"""Mgr Systems Engineering 1""","""COBO05""","""Full-Time""",null,"""0811 SP""","""Bachelors""","""[]""","""[]""","""[]""","""Manager""","""154000.00""","""en""",null,"""United States""","""2023-06-05 00:00:00""","""2023-06-08 08:27:08""","""2023""",9,"[""154000.00"", ""154000.00""]","[154000.0, 154000.0]",154000.0
"""Relay Engineer""","""Centennial, Colorado, United S…","""Unclear""",null,"""Electric Power Systems""","""Bachelors""","""[]""","""[]""","""[]""",null,"""105000.00""","""en-us""","""Centennial""","""United States""","""2023-05-10 04:00:00""","""2023-06-04 16:17:44""","""2023""",9,"[""105000.00"", ""105000.00""]","[105000.0, 105000.0]",105000.0
"""Web Engineer""",null,"""Full-Time""",true,"""Calbright College""",null,"""[WordPress, Ruby, Vue.js, Spri…","""[Hosting, Advertising, OSS, SE…","""[Education, Higher Education]""",null,"""123649.825""","""en""",null,null,"""2022-03-23 22:26:39""","""2023-05-20 06:28:17""","""2023""",10,"[""123649.825"", ""123649.825""]","[123649.825, 123649.825]",123649.825
"""Chief Engineer at Large""",null,"""Full-Time""",null,"""All Opportunities""","""Bachelors""","""[]""","""[]""","""[]""","""Chief""","""190000.00""","""en""","""Andover""","""United States""","""2023-04-28 16:19:43""","""2023-06-07 18:57:00""","""2023""",9,"[""190000.00"", ""190000.00""]","[190000.0, 190000.0]",190000.0
"""Industrial Engineer""","""Allflight HeadQuarters - Kent,…","""Unclear""",null,"""Infinity Air""","""Bachelors""","""[Microsoft]""","""[]""","""[Mechanical & Industrial Engin…",null,"""66000.00""","""en""","""Kent""","""United States""","""2023-01-16 00:00:00""","""2023-05-30 23:38:22""","""2023""",8,"[""66000.00"", ""66000.00""]","[66000.0, 66000.0]",66000.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""React native developer""","""Bangalore""","""Full-Time""",null,"""Athreya Business Consultancy""",null,"""[TypeScript, React, React Nati…","""[Templating Languages, Program…","""[Software]""","""Junior IC""","""400000.00""","""en""","""Bangalore""","""India""","""2024-03-11 00:00:00""","""2024-03-25 05:01:18""","""2024""",9,"[""400000.00"", ""400000.00""]","[400000.0, 400000.0]",400000.0
"""Senior Full Stack Software Eng…","""Remote""","""Full-Time""",true,"""Mogo Finance Technology""","""Bachelors""","""[S3, Ruby, AWS, TypeScript, Re…","""[Data Access & Databases, Soft…","""[Financial Services, Software]""","""Senior IC""","""130000.0""","""en""","""Vancouver""","""Canada""","""2024-03-28 00:00:00""","""2024-03-29 23:31:16""","""2024""",8,"[""130000.0"", ""130000.0""]","[130000.0, 130000.0]",130000.0
"""Mechanical Engineer""","""Richland, WA, 99354""",null,null,"""Craig Technologies""",null,"""[]""","""[]""","""[Mechanical & Industrial Engin…",null,"""89838.00""","""en-us""","""Richland""","""United States""",null,"""2024-03-21 01:14:41""","""2024""",8,"[""89838.00"", ""89838.00""]","[89838.0, 89838.0]",89838.0


In [14]:
whole_df.filter(pl.col("len_comp")>10, pl.col("comp_est").str.contains("[123456789]+[0]+[123456789]+[0]+")).with_columns(est=pl.col("comp_est").str.extract_all("[123456789]+[0]+")).filter(pl.col("est").list.len()>2).head(7)
#skipping cleaning those records as there are only 8 of them that would benefit from creating a logic to combine the first twoo or the last two strings into respectively lower or higher bound of the compensation estimation. I'll proceed only with the records that have only the lower and higher limit.

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,u32,list[str],list[f64],f64
"""Professional-System Engineer""","""Texas-Dallas""","""Full-Time""",null,"""Working at AT&T""",null,"""[Microsoft, Docker, Kubernetes…","""[Orchestration & Management, I…","""[]""","""Unclear Seniority""","""105000196000.00""","""en""","""Dallas""","""United States""","""2023-05-10 00:00:00""","""2023-05-24 11:22:53""","""2023""",15,"[""10"", ""5000"", ""196000""]","[10.0, 5000.0, 196000.0]",67003.333333
"""Systems Engineer, Commercial -…","""New York, NY""",null,null,"""Pure Storage""",null,"""[AWS, Gartner Magic Quadrant, …","""[Infrastructure, Cloud Native …","""[Data Storage]""",null,"""130200196000.00""","""en""","""New York City""","""United States""","""2023-05-08 00:00:00""","""2023-05-31 19:03:35""","""2023""",15,"[""130"", ""200"", ""196000""]","[130.0, 200.0, 196000.0]",65443.333333
"""Systems Engineer II""","""Goleta, CA, USA ● Plano, TX, U…","""Full-Time""",null,"""Inogen""","""Bachelors""","""[]""","""[]""","""[Manufacturing]""",null,"""102026.45000000001""","""en-us""","""Goleta""","""United States""","""2023-04-06 00:00:00""","""2023-05-20 23:56:56""","""2023""",18,"[""10"", ""20"", ""4500000000""]","[10.0, 20.0, 4.5000e9]",1.5000e9
"""Mechanical Maintenance Enginee…","""United States""",null,null,"""Rio Tinto Group""",null,"""[Excel, SAP]""","""[Servers & Compute, Cloud Mana…","""[]""",null,"""72960100320.00""","""en""",null,"""United States""","""2023-01-10 00:00:00""","""2023-06-09 03:17:08""","""2023""",14,"[""72960"", ""100"", ""320""]","[72960.0, 100.0, 320.0]",24460.0
"""Senior Software Development En…","""US.NE.Remote""","""Full-Time""",true,"""US05""","""Bachelors""","""[C#, Git, Tanzu, Rundeck, .NET…","""[Hadoop, Full Stack Framework,…","""[Software, Software Developmen…","""Senior IC""","""110160.76500000001""","""en""",null,"""United States""","""2023-05-22 00:00:00""","""2023-06-08 18:02:16""","""2023""",18,"[""110"", ""160"", ""7650000000""]","[110.0, 160.0, 7.6500e9]",2.5500e9
"""Process Project Engineer""","""Martinez, CA""","""Full-Time""",null,"""Martinez Refining Company""",null,"""[]""","""[]""","""[]""",null,"""130820.79000000001""","""en""","""Martinez""","""United States""","""2023-03-14 00:00:00""","""2023-05-26 02:25:26""","""2023""",18,"[""130"", ""820"", ""7900000000""]","[130.0, 820.0, 7.9000e9]",2.6333e9
"""Assistant Engineer""","""Poughkeepsie""","""Full-Time""",null,"""Central Hudson Gas & Electric …","""Bachelors""","""[Microsoft, Excel]""","""[Enterprise Customers, Back Of…","""[]""",null,"""78900110400.00""","""en""","""Poughkeepsie""","""United States""","""2024-02-02 00:00:00""","""2024-03-29 10:18:08""","""2024""",14,"[""78900"", ""110"", ""400""]","[78900.0, 110.0, 400.0]",26470.0


b) currency

In [15]:
whole_df["country"].value_counts(sort=True).head(10)

country,count
str,u32
"""United States""",44581
"""India""",10276
null,9708
"""United Kingdom""",4191
"""Germany""",3560
"""Canada""",2362
"""Brazil""",1783
"""France""",1530
"""Australia""",1340


In [16]:
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
whole_df = whole_df.with_columns(
    comp_dol = pl.when(pl.col("country") == "India")
        .then(pl.col("compensation")/83.4)
        .otherwise(pl.col("compensation"))
)

In [18]:
whole_df

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,u32,list[str],list[f64],f64,f64
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"""[Docker, GraphQL, NoSQL, IBM, …","""[Container Orchestration, Quer…","""[Software]""",null,"""""","""pl""","""Praha""","""Czech Republic""","""2023-03-13 05:12:29""","""2023-06-05 13:43:49""","""2023""",0,"["""", """"]","[null, null]",null,null
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"""[Sigma]""","""[Tools, Serverless]""","""[Manufacturing]""",null,"""""","""en""","""Sterling Heights""","""United States""","""2021-10-09 00:00:00""","""2023-05-24 05:35:57""","""2023""",0,"["""", """"]","[null, null]",null,null
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""","""[]""","""[]""","""[Design]""","""Senior IC""","""""","""en""","""Mishawaka""","""United States""","""2023-04-28 03:04:28""","""2023-05-19 14:48:10""","""2023""",0,"["""", """"]","[null, null]",null,null
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","""[AWS, Qualys, Splunk]""","""[Compute, Logging & Monitoring…","""[Cybersecurity, Security]""",null,"""""","""en""","""Herndon""","""United States""","""2023-04-03 00:00:00""","""2023-05-28 11:47:09""","""2023""",0,"["""", """"]","[null, null]",null,null
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"""[Objective-C, Subversion, Swif…","""[Cloud Native Storage, Program…","""[Software]""",null,"""""","""en""","""Kolkata""","""India""","""2017-01-17 11:35:48""","""2023-05-30 11:51:08""","""2023""",0,"["""", """"]","[null, null]",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Fire Protection Engineer (3+ y…","""1455 Frazee Rd, San Diego, CA …","""Full-Time""",false,"""Coffman Engineers""",null,"""[Facebook, Instagram, LinkedIn…","""[Tracking, Social Share, Socia…","""[Consulting]""",null,"""97500.00""","""en""","""San Diego""","""United States""","""2024-01-08 16:44:53""","""2024-04-01 21:24:50""","""2024""",8,"[""97500.00"", ""97500.00""]","[97500.0, 97500.0]",97500.0,97500.0
"""Mechanical Engineering Intern""",null,"""Intern""",null,"""Study and Work""",null,"""[]""","""[]""","""[Mechanical & Industrial Engin…","""Intern""",null,"""en""",null,null,"""2024-03-12 00:00:00""","""2024-03-27 17:53:22""","""2024""",null,"[null, null]","[null, null]",null,null
"""Automation Engineer (m/f/d)""","""Puurs""",null,null,"""Novartis""",null,"""[]""","""[]""","""[]""",null,null,"""fr""","""Puurs""","""Belgium""",null,"""2024-03-16 21:47:44""","""2024""",null,"[null, null]","[null, null]",null,null


In [19]:
whole_df.group_by("country").agg(pl.col("compensation").mean())

country,compensation
str,f64
"""Cuba""",150336.1875
"""Israel""",50819.526316
"""Australia""",81917.478538
"""Belarus""",1.0
"""Zimbabwe""",null
…,…
"""Uruguay""",null
"""Sudan""",null
"""Switzerland""",68037.75


In [20]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Mexico")
        .then(pl.col("comp_dol")/17)
        .otherwise(pl.col("comp_dol"))
)

In [21]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Poland")
        .then(pl.col("comp_dol")/4)
        .otherwise(pl.col("comp_dol"))
)

In [22]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Japan")
        .then(pl.col("comp_dol")/158)
        .otherwise(pl.col("comp_dol"))
)

In [23]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Colombia")
        .then(pl.col("comp_dol")/3_872) 
        .otherwise(pl.col("comp_dol"))
)

In [24]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "South Africa")
        .then(pl.col("comp_dol")/18.4) 
        .otherwise(pl.col("comp_dol"))
)


In [25]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Indonesia")
        .then(pl.col("comp_dol")/16_237) #16.237
        .otherwise(pl.col("comp_dol"))
)

In [26]:
whole_df = whole_df.with_columns(
    pl.when(pl.col("country") == "Kyrgyzstan")
        .then(pl.col("comp_dol")/87.7) #16.237
        .otherwise(pl.col("comp_dol"))
)

c) hour rates -> monthly estimation

In [27]:
whole_df.filter(pl.col("comp_dol")>0, pl.col("comp_dol")<200).select(pl.col("comp_dol")).plot(kind="hist")

:Histogram   [comp_dol]   (comp_dol_count)

In [28]:
whole_df = whole_df.with_columns(
    pl.when((pl.col("comp_dol")>0) & (pl.col("comp_dol")<200))
        .then(pl.col("comp_dol")*40*4.5*12)
        .otherwise(pl.col("comp_dol"))
)

In [29]:
whole_df.filter(pl.col("comp_dol")>0, pl.col("comp_dol")<5_000_000).select(pl.col("comp_dol")).plot(kind="hist")

:Histogram   [comp_dol]   (comp_dol_count)

In [30]:
whole_df.filter(pl.col("comp_dol")>1000, pl.col("comp_dol")<10_000).select(pl.col("comp_dol")).plot(kind="hist")

:Histogram   [comp_dol]   (comp_dol_count)

In [31]:
whole_df = whole_df.with_columns(
    pl.when((pl.col("comp_dol")>1000) & (pl.col("comp_dol")<10_000))
        .then(pl.col("comp_dol")*12)
        .otherwise(pl.col("comp_dol"))
)
#assuming here that the amounts between 1000-10000 are monthly estimates

In [32]:
comp_imput = whole_df.filter(pl.col("comp_dol")>0, pl.col("comp_dol")<1_000_000).select("comp_dol").mean()
#mean value of compensation 

In [33]:
whole_df = whole_df.with_columns(
    comp_dol = pl.when(pl.col("comp_dol")>10_000_000)
        .then(comp_imput)
        .otherwise(pl.col("comp_dol")))

In [34]:
whole_df.filter(pl.col("comp_dol")<1_000_000).select(pl.col("comp_dol")).plot(kind="hist")

:Histogram   [comp_dol]   (comp_dol_count)

2. Converting 'job_published_at', 'last_indexed' to Date

In [35]:
whole_df = whole_df.with_columns(
    pl.col("job_published_at", "last_indexed").str.to_datetime().cast(pl.Date)
)

3. Converting 'tags_matched', 'tag_categories', 'categories' from str to list[str] and to one col with tags

In [36]:
def string_to_nested(df, cols):
    """
    takes a df and list of columns that contain strings with lists
    and turns them into nested datatype List
    """
    for col in cols:
        df = df.with_columns(
            pl.col(col).str.extract_all(r"\w+").cast(pl.List(pl.String))
        )
    return df

In [37]:
cols_to_change = ["tags_matched", "tag_categories", "categories"]
whole_df = string_to_nested(whole_df, cols_to_change)

In [38]:
whole_df = whole_df.with_columns(
    all_tags=pl.col("tags_matched")
    .list.concat(["tag_categories", "categories"])
    .list.unique()
)
whole_df["all_tags"]

all_tags
list[str]
"[""Science"", ""TypeScript"", … ""Programming""]"
"[""Tools"", ""Serverless"", … ""Sigma""]"
"[""Design""]"
"[""PaaS"", ""Infrastructure"", … ""Security""]"
"[""iOS"", ""Software"", … ""Version""]"
…
"[""Shortening"", ""Twitter"", … ""Link""]"
"[""Industrial"", ""Engineering"", ""Mechanical""]"
[]


4. Language/ education/hours/seniority -> pl.Categorical

a) language - cleaning the string, imputing the null value and casting to pl.Categorical

In [39]:
whole_df = whole_df.with_columns(pl.col("language").str.head(2))

In [40]:
whole_df = whole_df.with_columns(
    pl.col("language").fill_null("unknown").cast(pl.Categorical)
)  # introducing "unknow" category, only 584 nulls

/tmp/ipykernel_24935/2234765048.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  whole_df = whole_df.with_columns(


In [41]:
whole_df.filter(pl.col("language")=="unknown")

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Senior Lab Engineer - Robotics…","""UrbanVault 1608, 7th Cross Opp…","""Full-Time""",null,"""CynLr""",null,null,null,"[""Robotics""]","""Senior IC""","""""","""unknown""","""Karnataka""","""India""",2023-05-15,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,null
"""computer engineer""",null,"""Full-Time""",null,"""LAMAS consulting""",null,[],[],"[""Consulting""]",null,"""""","""unknown""","""Riyadh""","""Saudi Arabia""",null,2023-06-07,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Consulting""]"
"""Full-stack web developer""","""Singapore Office""",null,null,"""LittleLives""",null,null,null,"[""Web"", ""Development"", ""Software""]",null,"""""","""unknown""","""Singapore""","""Singapore""",2023-06-03,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,null
"""FEA Physicist/Engineer""","""Aldermaston""","""Full-Time""",null,"""AWE""",null,null,null,"[""Science""]",null,"""""","""unknown""",null,"""United Kingdom""",2023-05-02,2023-05-25,"""2023""",0,"["""", """"]","[null, null]",null,null,null
"""Go Engineer""",null,null,null,"""Rackspace""",null,null,null,[],null,"""""","""unknown""","""San Francisco""","""United States""",null,2023-05-21,"""2023""",0,"["""", """"]","[null, null]",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Senior Commercial Strategy Man…","""Munich, Bavaria, DEU""","""Full-Time""",null,"""Amazon Deutschland Services Gm…",null,null,null,"[""Transportation""]","""Manager""","""""","""unknown""","""Munich""","""Germany""",2024-03-07,2024-04-01,"""2024""",0,"["""", """"]","[null, null]",null,null,null
"""Solutions Architect Intern""","""Toronto, Ontario, CAN""","""Full-Time""",null,"""Amazon Development Centre Cana…",null,null,null,"[""Architecture"", ""Planning""]","""Intern""","""""","""unknown""","""Toronto""","""Canada""",2024-03-12,2024-04-01,"""2024""",0,"["""", """"]","[null, null]",null,null,null
"""Principal Product Manager, Pla…","""Tokyo, JPN""","""Full-Time""",null,"""Amazon Japan G.K.""",null,null,null,[],"""Staff IC""","""""","""unknown""","""Tokyo""","""Japan""",2024-01-23,2024-03-29,"""2024""",0,"["""", """"]","[null, null]",null,null,null


b) education - imputing null values

In [42]:
whole_df["education"].value_counts().head(10)

education,count
str,u32
"""High School""",3509
"""Some College""",19
"""No Education Requirement""",16
"""Vocational""",169
"""Associates""",149
null,62098
"""Certification""",1
"""Masters""",1587
"""Some High School""",921


In [43]:
whole_df.filter(pl.col("education").is_not_null(), pl.col("compensation")>0)
#since about a half of compensation records contain information about education I'll leave this column and will introduce "unknown" category

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Mgr Systems Engineering 1""","""COBO05""","""Full-Time""",null,"""0811 SP""","""Bachelors""",[],[],[],"""Manager""","""154000.00""","""en""",null,"""United States""",2023-06-05,2023-06-08,"""2023""",9,"[""154000.00"", ""154000.00""]","[154000.0, 154000.0]",154000.0,154000.0,[]
"""Relay Engineer""","""Centennial, Colorado, United S…","""Unclear""",null,"""Electric Power Systems""","""Bachelors""",[],[],[],null,"""105000.00""","""en""","""Centennial""","""United States""",2023-05-10,2023-06-04,"""2023""",9,"[""105000.00"", ""105000.00""]","[105000.0, 105000.0]",105000.0,105000.0,[]
"""Chief Engineer at Large""",null,"""Full-Time""",null,"""All Opportunities""","""Bachelors""",[],[],[],"""Chief""","""190000.00""","""en""","""Andover""","""United States""",2023-04-28,2023-06-07,"""2023""",9,"[""190000.00"", ""190000.00""]","[190000.0, 190000.0]",190000.0,190000.0,[]
"""Industrial Engineer""","""Allflight HeadQuarters - Kent,…","""Unclear""",null,"""Infinity Air""","""Bachelors""","[""Microsoft""]",[],"[""Mechanical"", ""Industrial"", ""Engineering""]",null,"""66000.00""","""en""","""Kent""","""United States""",2023-01-16,2023-05-30,"""2023""",8,"[""66000.00"", ""66000.00""]","[66000.0, 66000.0]",66000.0,66000.0,"[""Mechanical"", ""Industrial"", … ""Microsoft""]"
"""Database Engineer Lead""","""New York, NY - 399 Park Avenue""","""Full-Time""",null,"""MPG Operations""","""Bachelors""","[""Vertica"", ""Ansible"", … ""MongoDB""]","[""Big"", ""Data"", … ""Accounting""]","[""Data"", ""Storage""]",null,"""212500.00""","""en""",null,"""United States""",2023-04-12,2023-05-26,"""2023""",9,"[""212500.00"", ""212500.00""]","[212500.0, 212500.0]",212500.0,212500.0,"[""MongoDB"", ""Infrastructure"", … ""Config""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Public Safety Officer - Secur…","""Chino, California, United Stat…","""Part-Time""",null,"""Prime Healthcare""","""High School""","[""IBM""]","[""Infrastructure"", ""Analytics"", … ""IaaS""]","[""Public"", ""Safety"", … ""Security""]","""Unclear Seniority""","""20.305""","""en""","""Chino""","""United States""",2024-03-08,2024-04-01,"""2024""",6,"[""20.305"", ""20.305""]","[20.305, 20.305]",20.305,43858.8,"[""Security"", ""Safety"", … ""Analytics""]"
"""Product Manager""","""San Francisco, California, Uni…","""Full-Time""",false,null,"""Masters""","[""Salesforce""]","[""Enterprise"", ""Customers"", … ""Customers""]",[],"""IC""","""90000.00""","""en""","""San Francisco""","""United States""",2024-03-08,2024-03-14,"""2024""",8,"[""90000.00"", ""90000.00""]","[90000.0, 90000.0]",90000.0,90000.0,"[""Customers"", ""Enterprise"", … ""CRM""]"
"""Security Field Supervisor Arme…","""Los Angeles, California, Unite…","""Full-Time""",null,"""Allied Universal""","""High School""",[],[],"[""Facilities"", ""Services"", … ""Dentistry""]","""Manager""","""52880.00""","""en""","""Los Angeles""","""United States""",2024-03-28,2024-03-29,"""2024""",8,"[""52880.00"", ""52880.00""]","[52880.0, 52880.0]",52880.0,52880.0,"[""Services"", ""Insurance"", … ""Dentistry""]"


In [44]:
whole_df.filter(pl.col("education").is_not_null(), pl.col("seniority")=="Junior IC")
#since about one third of junior roles contain information about education I'll leave this column and will introduce "unknown" category

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Electrical Engineer – Entry Le…","""Ann Arbor""","""Full-Time""",null,"""Barr Engineering Co.""","""Bachelors""","[""Excel"", ""FactoryTalk""]","[""SMB"", ""Customers"", … ""A""]",[],"""Junior IC""","""""","""en""","""Ann Arbor""","""United States""",2023-04-19,2023-05-25,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Customers"", ""Code"", … ""Biz""]"
"""Electrical Engineer – Entry Le…","""Minneapolis""","""Full-Time""",null,"""Barr Engineering Co.""","""Bachelors""","[""Excel""]","[""Midsize"", ""Customers"", … ""Spreadsheets""]",[],"""Junior IC""","""""","""en""","""Minneapolis""","""United States""",2023-04-06,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""SMB"", ""Midsize"", … ""Misc""]"
"""Master@IBM – Junior Security C…","""Zurich, Switzerland""","""Part-Time""",null,"""IBM""","""Bachelors""","[""CyberArk"", ""IBM"", … ""Blockchain""]","[""OSS"", ""Analytics"", … ""Platforms""]","[""IT"", ""Security"", ""Consulting""]","""Junior IC""","""""","""zh""","""Zurich""","""Switzerland""",2023-04-04,2023-05-23,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Cloud"", ""Security"", … ""KCSP""]"
"""Software Developer- Entry Leve…",null,"""Full-Time""",false,null,"""Some High School""",[],[],"[""Software""]","""Junior IC""","""""","""en""","""Providence""","""United States""",2022-04-27,2023-05-28,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Software""]"
"""JUNIOR DATA SCIENTIST - Dubai,…","""Dubai, UAE""","""Full-Time""",null,"""Dubai, UAE.""","""Bachelors""","[""Python"", ""Matplotlib"", … ""seaborn""]","[""Data"", ""Science"", … ""SQL""]",[],"""Junior IC""","""60000.00""","""en""","""Prague""","""Czech Republic""",2023-04-24,2023-06-08,"""2023""",8,"[""60000.00"", ""60000.00""]","[60000.0, 60000.0]",60000.0,60000.0,"[""seaborn"", ""Science"", … ""Stat""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Entry Level Structural Enginee…","""Charlotte, NC""",null,null,"""LJB""","""Bachelors""",[],[],"[""Mechanical"", ""Industrial"", ""Engineering""]","""Junior IC""",null,"""en""","""Charlotte""","""United States""",2024-03-01,2024-03-27,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Industrial"", ""Engineering"", ""Mechanical""]"
"""Entry Level Civil/Environmenta…","""Detroit, MI""","""Full-Time""",null,"""Wade Trim""","""Bachelors""",[],[],[],"""Junior IC""",null,"""en""","""Detroit""","""United States""",2023-09-11,2024-03-16,"""2024""",null,"[null, null]","[null, null]",null,null,[]
"""Test Engineer, Junior to Inter…","""Aberdeen, MD 21001""","""Full-Time""",null,"""Veritech""","""Bachelors""","[""Linux""]","[""OS""]",[],"""Junior IC""",null,"""en""","""Aberdeen""","""United States""",2024-01-11,2024-03-30,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Linux"", ""OS""]"


In [45]:
whole_df = whole_df.with_columns(
    pl.col("education").fill_null("unknown").cast(pl.Categorical)
)  # introducing "unknow" category 
#consider skipping this column as there are 62098 null values

/tmp/ipykernel_24935/984726698.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  whole_df = whole_df.with_columns(


c) hours - imputing null values

In [46]:
whole_df = whole_df.with_columns(
    pl.col("hours").fill_null("Unclear").cast(pl.Categorical)
)  # introducing "unknow" category in 27_739 null records

/tmp/ipykernel_24935/4062927793.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  whole_df = whole_df.with_columns(


In [47]:
whole_df = whole_df.with_columns(
    pl.col("remote").fill_null("unknown").cast(pl.Categorical)
)  # introducing "unknow" category in 27_739 null records

/tmp/ipykernel_24935/3633386334.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  whole_df = whole_df.with_columns(


In [48]:
whole_df.filter(pl.col("job_location").str.contains("(?i)not remote"), pl.col("remote")!="true")

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Science & Engineering Circulat…","""Not Remote""","""Student""","""unknown""","""UW-Madison""","""unknown""","[""Facebook"", ""AMP""]","[""Store"", ""Design"", … ""AMP""]","[""Healthcare""]",null,"""""","""en""",null,null,2023-05-16,2023-05-31,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Advertising"", ""Facebook"", … ""Sales""]"
"""Transportation Engineering Int…","""Not Remote""","""Student""","""unknown""","""UW-Madison""","""unknown""","[""Facebook""]","[""Advertising"", ""Social"", ""Media""]","[""Transportation""]","""Intern""","""""","""en""",null,null,2023-04-14,2023-05-31,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Transportation"", ""Facebook"", … ""Advertising""]"
"""High School Youth Program Inst…","""Not Remote""","""Student""","""unknown""","""UW-Madison""","""High School""",[],[],"[""Primary"", ""Secondary"", … ""Education""]",null,"""4650.00""","""en""",null,null,2023-12-11,2024-03-24,"""2024""",7,"[""4650.00"", ""4650.00""]","[4650.0, 4650.0]",4650.0,55800.0,"[""Primary"", ""Design"", … ""Education""]"


In [49]:
whole_df.filter(pl.col("job_location").str.contains("(?i)remote"), pl.col("remote")!="true")

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Product Manager - E-Sports/Gam…","""Eastern Canada|Ontario|Montrea…","""Full-Time""","""unknown""","""Targeted Talent""","""unknown""",[],[],[],"""IC""","""""","""en""","""Montreal""","""Canada""",2021-04-23,2023-05-25,"""2023""",0,"["""", """"]","[null, null]",null,null,[]
"""Software Solution Consultant""","""GBR_Remote""","""Full-Time""","""unknown""","""UK AX""","""Bachelors""","[""Microsoft"", ""Linux"", ""C""]","[""Programming"", ""Languages"", … ""OSS""]","[""Consulting"", ""Software""]","""Contract""","""""","""en""",null,"""United Kingdom""",2023-04-19,2023-05-26,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""OS"", ""Consulting"", … ""OSS""]"
"""Software Development Engineer …","""IN.Bangalore.Remote""","""Full-Time""","""unknown""","""IN50""","""Bachelors""","[""Twitter"", ""Facebook"", … ""js""]","[""Stat"", ""Tools"", … ""Share""]","[""Software"", ""Development"", … ""Software""]",null,"""""","""en""","""Bengaluru""","""India""",2023-05-25,2023-05-26,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Development"", ""UI"", … ""Related""]"
"""Founding Engineer""","""New York|Remote|Hybrid""","""Full-Time""","""unknown""","""NovumTech Partners""","""unknown""",[],[],[],null,"""""","""en""",null,"""United States""",2023-02-02,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,[]
"""Science & Engineering Circulat…","""Not Remote""","""Student""","""unknown""","""UW-Madison""","""unknown""","[""Facebook"", ""AMP""]","[""Store"", ""Design"", … ""AMP""]","[""Healthcare""]",null,"""""","""en""",null,null,2023-05-16,2023-05-31,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Advertising"", ""Facebook"", … ""Sales""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Licensed Civil Engineer - Data…","""Remote, Omaha, NE, us""","""Full-Time""","""false""","""Olsson""","""Bachelors""",[],[],"[""Civil"", ""Engineering"", … ""Infrastructure""]",null,null,"""en""","""Omaha""","""United States""",2024-02-22,2024-04-02,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Network"", ""Civil"", … ""Infrastructure""]"
"""Lead Cloud Infrastructure / De…","""US-NH-Portsmouth | US-Remote |…","""Full-Time""","""unknown""","""Unavailable""","""Bachelors""","[""Terraform"", ""Ansible"", … ""Docker""]","[""DevOps"", ""Automation"", … ""Software""]","[""DevOps""]",null,null,"""en""","""Portsmouth""","""United States""",2022-04-01,2024-04-01,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Software"", ""Data"", … ""Cloud""]"
"""High School Youth Program Inst…","""Not Remote""","""Student""","""unknown""","""UW-Madison""","""High School""",[],[],"[""Primary"", ""Secondary"", … ""Education""]",null,"""4650.00""","""en""",null,null,2023-12-11,2024-03-24,"""2024""",7,"[""4650.00"", ""4650.00""]","[4650.0, 4650.0]",4650.0,55800.0,"[""Primary"", ""Design"", … ""Education""]"


In [50]:
whole_df.filter(pl.col("job_name").str.contains("remote"), pl.col("remote")!="true")

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Data Engineer (ElasticSearch) …",null,"""Unclear""","""unknown""","""TechINT Solutions Group""","""unknown""","[""JSON"", ""Azure"", … ""Logstash""]","[""Observability"", ""Search"", … ""Search""]","[""Airlines"", ""Aerospace"", ""Security""]",null,"""""","""en""",null,null,null,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""CI"", ""Integration"", … ""Provisioning""]"
"""Staff Engineer - Paris, Montpe…","""France""","""Full-Time""","""unknown""","""Swile""","""unknown""","[""Kotlin"", ""Node"", … ""AWS""]","[""Libraries"", ""IaaS"", … ""Configuration""]",[],"""Staff IC""","""""","""en""",null,"""France""",2022-08-30,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Languages"", ""Player"", … ""Libraries""]"
"""Information Systems Security O…","""Beavercreek, Ohio""","""Unclear""","""unknown""","""Riverside Research""","""unknown""","[""ArcSight""]","[""Security""]","[""Security""]","""Unclear Seniority""","""""","""en""","""Beavercreek""","""United States""",2023-03-16,2023-06-06,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""ArcSight"", ""Security""]"
"""Senior Data Scientist -remote""",null,"""Contract""","""false""",null,"""Bachelors""","[""SQL"", ""SAS"", ""Python""]","[""SQL"", ""General"", … ""Tools""]",[],"""Senior IC""","""""","""en""","""San Diego""","""United States""",2023-05-11,2023-05-26,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""General"", ""Science"", … ""Business""]"
"""Software Developer III/IV (hyb…",null,"""Unclear""","""unknown""","""LeidosBioMed""","""unknown""",[],[],"[""Software"", ""BioTech""]",null,null,"""en""","""Frederick""","""United States""",2023-05-19,2023-06-07,"""2023""",null,"[null, null]","[null, null]",null,null,"[""BioTech"", ""Software""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Cloud Architect (hybrid-remote…","""Lansing, MI, United States""","""Unclear""","""unknown""","""Nanosoft Consulting Talent Pag…","""unknown""","[""Azure"", ""DevOps"", … ""Bus""]","[""Project"", ""Management"", … ""Queues""]","[""Architecture"", ""Planning"", ""Consulting""]",null,null,"""en""","""Lansing""","""United States""",null,2024-03-29,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Compute"", ""Products"", … ""Planning""]"
"""Werkstudent im Bereich Projekt…","""Daimlerstraße 6, 71229 Leonber…","""Part-Time""","""false""","""Bosch Group""","""Bachelors""",[],[],"[""Security""]",null,null,"""de""","""Leonberg""","""Germany""",2024-03-22,2024-03-30,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Security""]"
"""Junior Product Owner - Ecommer…","""Kraków""","""Unclear""","""unknown""",null,"""unknown""","[""Mastercard""]",[],"[""eCom""]","""Junior IC""",null,"""pl""","""Kraków""","""Poland""",2024-03-26,2024-03-28,"""2024""",null,"[null, null]","[null, null]",null,null,"[""eCom"", ""Mastercard""]"


In [51]:
whole_df = whole_df.with_columns(
    remote = pl.when((pl.col("job_name").str.contains("remote")) | (pl.col("job_location").str.contains("(?i)remote")))
        .then(pl.lit("true"))
        .otherwise(pl.col("remote"))
)

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [52]:
whole_df.filter(pl.col("job_name").str.contains("remote"), pl.col("remote")!="true")

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],str,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]


In [53]:
whole_df["seniority"].value_counts(sort=True)

seniority,count
str,u32
null,53579
"""Senior IC""",17840
"""Manager""",6688
"""IC""",6623
"""Unclear Seniority""",4593
…,…
"""Chief""",557
"""Exec""",465
"""Senior Manager""",273


In [54]:
whole_df = whole_df.with_columns(
    pl.col("seniority").fill_null("Unclear Seniority").cast(pl.Categorical)
)  # filling nulls with "Unclear Seniority" 

/tmp/ipykernel_24935/3224810329.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  whole_df = whole_df.with_columns(


In [55]:
whole_df.filter(pl.col("job_name").str.contains("[a-zA-Z]+\."))

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""Sr. Planning and Hydraulic Mod…","""United States""","""Unclear""","""unknown""","""Barge Design Solutions""","""unknown""",[],[],"[""Design""]","""Senior IC""",null,"""en""",null,"""United States""",2023-05-02,2023-06-09,"""2023""",null,"[null, null]","[null, null]",null,null,"[""Design""]"
"""Sr. Architect / Engineering Ma…","""Delhi, Delhi, 110017 India""","""Full-Time""","""unknown""","""Zenon Analytics""","""unknown""","[""Tableau"", ""Azure"", … ""BI""]","[""Databases"", ""Database"", … ""IaaS""]","[""Architecture"", ""Planning"", … ""Analytics""]","""Director""",null,"""en""","""Delhi""","""India""",2023-02-22,2023-06-09,"""2023""",null,"[null, null]","[null, null]",null,null,"[""IaaS"", ""Java"", … ""Programming""]"
"""Information Technology Jr. Pro…","""7700 Arlington Blvd, Falls Chu…","""Full-Time""","""true""","""Blue Iris Tech Consulting""","""Bachelors""","[""Iris"", ""AppDynamics"", ""Excel""]","[""DevOps"", ""APM"", … ""Spreadsheets""]","[""Security"", ""Consulting""]","""IC""","""""","""en""","""Falls Church""","""United States""",2022-06-18,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Enterprise"", ""No"", … ""Consulting""]"
"""SR. Software Engineer - Rust, …",null,"""Full-Time""","""true""",null,"""Bachelors""","[""SQL"", ""Oracle"", … ""Snowflake""]","[""Datastores"", ""SaaS"", … ""Databases""]","[""Software""]","""Unclear Seniority""","""""","""en""","""San Diego""","""United States""",2022-03-16,2023-05-26,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Languages"", ""Warehouses"", … ""Big""]"
"""Sr. Industrial Engineer - Ware…","""St Louis, MO""","""Unclear""","""unknown""","""CCA PROFESSIONAL RECRUITING""","""unknown""",[],[],"[""Mechanical"", ""Industrial"", … ""Warehousing""]","""Senior IC""","""50047.5""","""en""","""St Louis""","""United States""",null,2023-06-05,"""2023""",7,"[""50047.5"", ""50047.5""]","[50047.5, 50047.5]",50047.5,50047.5,"[""Staffing"", ""Recruiting"", … ""Engineering""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""J.P. Morgan Wealth Management …","""Wilmington, DE, United States""","""Full-Time""","""unknown""","""260312-SOUTH FLORIDA REGION AD…","""Bachelors""","[""Salesforce"", ""Jira""]","[""Midsize"", ""Customers"", … ""Management""]",[],"""Senior IC""",null,"""en""","""Wilmington""","""United States""",2024-03-13,2024-03-14,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Midsize"", ""Issue"", … ""Jira""]"
"""Managed Svcs Systems Asc. Engi…","""Enschede""","""Full-Time""","""unknown""","""Virtu Secure Web Services""","""unknown""","[""Equinix""]",[],[],"""Unclear Seniority""",null,"""en""","""Enschede""","""Netherlands""",2023-10-18,2024-03-14,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Equinix""]"
"""Sr. Information Assurance/Secu…","""Remote from Home - US - San An…","""Full-Time""","""true""","""Feditc""","""Bachelors""",[],[],"[""Security""]","""Senior IC""",null,"""en""","""San Antonio""","""United States""",2023-09-06,2024-03-28,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Security""]"


In [56]:
whole_df = whole_df.with_columns(pl.col("job_name").str.to_lowercase().str.replace("sr.", "senior").str.replace("jr.", "junior"))
#replacing the sr. jr. with senior and junior

In [57]:
whole_df = whole_df.with_columns(
    seniority = pl.when(pl.col("job_name").str.contains("senior"))
        .then(pl.lit("Senior IC"))
        .otherwise(pl.col("seniority"))
)
#there are job_names that contain both junior and senior offers, I choose to label them as Junior IC

In [58]:
whole_df = whole_df.with_columns(
    seniority = pl.when(pl.col("job_name").str.contains("junior"))
        .then(pl.lit("Junior IC"))
        .otherwise(pl.col("seniority"))
)

In [59]:
whole_df.filter(pl.col("job_name").str.contains("junior"), pl.col("seniority")!="Junior IC") #sanity checks if it worked

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]


In [60]:
whole_df.filter(pl.col("seniority")=="Junior IC") #sanity check for numbers

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str]
"""information technology junior …","""7700 Arlington Blvd, Falls Chu…","""Full-Time""","""true""","""Blue Iris Tech Consulting""","""Bachelors""","[""Iris"", ""AppDynamics"", ""Excel""]","[""DevOps"", ""APM"", … ""Spreadsheets""]","[""Security"", ""Consulting""]","""Junior IC""","""""","""en""","""Falls Church""","""United States""",2022-06-18,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Enterprise"", ""No"", … ""Consulting""]"
"""javascript engineer""","""Bucuresti, Sector 2, Romania""","""Full-Time""","""unknown""","""Atta Systems""","""unknown""","[""JavaScript"", ""Zoom"", … ""React""]","[""Collaboration"", ""Messaging"", … ""Libraries""]","[""Software"", ""Startups""]","""Junior IC""","""""","""en""","""Bucuresti""","""Romania""",2023-05-03,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""OSS"", ""Project"", … ""DevOps""]"
"""electrical engineer – entry le…","""Ann Arbor""","""Full-Time""","""unknown""","""Barr Engineering Co.""","""Bachelors""","[""Excel"", ""FactoryTalk""]","[""SMB"", ""Customers"", … ""A""]",[],"""Junior IC""","""""","""en""","""Ann Arbor""","""United States""",2023-04-19,2023-05-25,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Customers"", ""Code"", … ""Biz""]"
"""electrical engineer – entry le…","""Minneapolis""","""Full-Time""","""unknown""","""Barr Engineering Co.""","""Bachelors""","[""Excel""]","[""Midsize"", ""Customers"", … ""Spreadsheets""]",[],"""Junior IC""","""""","""en""","""Minneapolis""","""United States""",2023-04-06,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""SMB"", ""Midsize"", … ""Misc""]"
"""junior data analyst - corporat…","""PT - 11 - Lisbon""","""Full-Time""","""unknown""","""BNP Paribas""","""unknown""","[""Excel""]","[""Misc"", ""Biz"", … ""Customers""]","[""Financial"", ""Services"", ""Business""]","""Junior IC""","""""","""en""","""Lisbon""","""Portugal""",null,2023-05-28,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Midsize"", ""FP"", … ""Code""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""assistant engineer (process co…","""Coimbatore""","""Full-Time""","""unknown""","""Sandfits Foundries""","""unknown""",[],[],[],"""Junior IC""","""360000.00""","""en""","""Coimbatore""","""India""",2023-09-27,2024-03-22,"""2024""",9,"[""360000.00"", ""360000.00""]","[360000.0, 360000.0]",360000.0,51798.561151,[]
"""junior développeur front/back-…",null,"""Full-Time""","""unknown""","""Advize Group""","""unknown""","[""MySQL"", ""MongoDB"", … ""jQuery""]","[""Databases"", ""OSS"", … ""Database""]","[""Fintech""]","""Junior IC""",null,"""fr""",null,null,2020-12-09,2024-03-18,"""2024""",null,"[null, null]","[null, null]",null,null,"[""CDI"", ""jQuery"", … ""Datastores""]"
"""react native developer""","""Bangalore""","""Full-Time""","""unknown""","""Athreya Business Consultancy""","""unknown""","[""TypeScript"", ""React"", … ""Bootstrap""]","[""Templating"", ""Languages"", … ""Framework""]","[""Software""]","""Junior IC""","""400000.00""","""en""","""Bangalore""","""India""",2024-03-11,2024-03-25,"""2024""",9,"[""400000.00"", ""400000.00""]","[400000.0, 400000.0]",400000.0,57553.956835,"[""Development"", ""OSS"", … ""Software""]"


In [61]:
whole_df = whole_df.with_columns(tags = pl.col("all_tags").list.join(" "))
whole_df

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags,tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str],str
"""backend developer""","""Praha, Czech Republic""","""Unclear""","""unknown""","""IBM""","""unknown""","[""Docker"", ""GraphQL"", … ""Cloud""]","[""Container"", ""Orchestration"", … ""Databases""]","[""Software""]","""Unclear Seniority""","""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Science"", ""TypeScript"", … ""Programming""]","""Science TypeScript Horizontal …"
"""manufacturing engineer""",null,"""Full-Time""","""false""",null,"""unknown""","[""Sigma""]","[""Tools"", ""Serverless""]","[""Manufacturing""]","""Unclear Seniority""","""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Tools"", ""Serverless"", … ""Sigma""]","""Tools Serverless Manufacturing…"
"""design engineer, motorized pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""","""unknown""","""ABI Attachments""","""Bachelors""",[],[],"[""Design""]","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Design""]","""Design"""
"""cybersecurity engineer""",null,"""Full-Time""","""false""",null,"""Bachelors""","[""AWS"", ""Qualys"", ""Splunk""]","[""Compute"", ""Logging"", … ""Security""]","[""Cybersecurity"", ""Security""]","""Unclear Seniority""","""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""PaaS"", ""Infrastructure"", … ""Security""]","""PaaS Infrastructure Analytics …"
"""your career so choose wisely w…","""Kolkata, India""","""Full-Time""","""false""","""Adeeba e Services""","""unknown""","[""Objective"", ""C"", … ""Git""]","[""Cloud"", ""Native"", … ""Control""]","[""Software""]","""Unclear Seniority""","""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""iOS"", ""Software"", … ""Version""]","""iOS Software Runtime Storage P…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""fire protection engineer (3+ y…","""1455 Frazee Rd, San Diego, CA …","""Full-Time""","""false""","""Coffman Engineers""","""unknown""","[""Facebook"", ""Instagram"", … ""Twitter""]","[""Tracking"", ""Social"", … ""CDN""]","[""Consulting""]","""Unclear Seniority""","""97500.00""","""en""","""San Diego""","""United States""",2024-01-08,2024-04-01,"""2024""",8,"[""97500.00"", ""97500.00""]","[97500.0, 97500.0]",97500.0,97500.0,"[""Shortening"", ""Twitter"", … ""Link""]","""Shortening Twitter CDN Instagr…"
"""mechanical engineering intern""",null,"""Intern""","""unknown""","""Study and Work""","""unknown""",[],[],"[""Mechanical"", ""Industrial"", ""Engineering""]","""Intern""",null,"""en""",null,null,2024-03-12,2024-03-27,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Industrial"", ""Engineering"", ""Mechanical""]","""Industrial Engineering Mechani…"
"""automation engineer (m/f/d)""","""Puurs""","""Unclear""","""unknown""","""Novartis""","""unknown""",[],[],[],"""Unclear Seniority""",null,"""fr""","""Puurs""","""Belgium""",null,2024-03-16,"""2024""",null,"[null, null]","[null, null]",null,null,[],""""""


5. If country/city not present use job_location

In [62]:
whole_df.filter(pl.col("city").is_null(), pl.col("job_location").is_not_null())
#I could fuzzy match my df with cities with this column and get more data out of it, I'll leave it for reiteration - 16_1519 missing
whole_df.filter(pl.col("country").is_null(), pl.col("job_location").is_not_null())
#same holds for this one, missing country is only around 2_968

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags,tags
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str],str
"""staff software engineer - back…","""Remote""","""Full-Time""","""true""","""String Technology""","""unknown""","[""Terraform"", ""Redis"", … ""AWS""]","[""Timeseries"", ""Database"", … ""Databases""]","[""Software""]","""Staff IC""","""""","""en""",null,null,2023-02-23,2023-06-04,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Definition"", ""Memory"", … ""Access""]","""Definition Memory Timeseries C…"
"""senior silicon development eng…","""Korea, Republic of - KR - Buch…","""Unclear""","""unknown""",null,"""unknown""",[],[],[],"""Senior IC""","""""","""en""",null,null,2023-05-10,2023-06-04,"""2023""",0,"["""", """"]","[null, null]",null,null,[],""""""
"""senior engineer - geotech""","""Capetown""","""Full-Time""","""unknown""",null,"""unknown""",[],[],[],"""Senior IC""","""""","""en""",null,null,2023-05-19,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,[],""""""
"""senior software engineer - plc…","""PLC Technologies""","""Unclear""","""unknown""","""YASH Technologies""","""unknown""",[],[],"[""Software""]","""Senior IC""","""""","""en""",null,null,2023-05-19,2023-05-31,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Software""]","""Software"""
"""senior r&d systems engineer""","""Malibu, CA""","""Full-Time""","""unknown""","""HRL Laboratories""","""Bachelors""",[],[],[],"""Senior IC""","""""","""en""",null,null,2023-01-05,2023-05-26,"""2023""",0,"["""", """"]","[null, null]",null,null,[],""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mechanical design engineer""","""Sandall Stones Road""","""Unclear""","""unknown""","""Premier Group Recruitment""","""unknown""",[],[],"[""Recruiting"", ""Staffing"", ""Design""]","""Unclear Seniority""",null,"""en""",null,null,2024-02-23,2024-04-01,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Recruiting"", ""Design"", ""Staffing""]","""Recruiting Design Staffing"""
"""softwareentwickler rpg / ibm i…","""Hannover Heiligerstraße, Raven…","""Full-Time""","""unknown""",null,"""unknown""","[""IBM""]","[""Analytics"", ""Infrastructure"", … ""Platforms""]","[""Software""]","""Unclear Seniority""",null,"""en""",null,null,2024-03-19,2024-03-31,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Access"", ""IBM"", … ""Science""]","""Access IBM Analytics Governanc…"
"""senior siem support engineer (…","""DC, US""","""Unclear""","""unknown""","""Gray Tier Technologies""","""unknown""","[""Splunk"", ""IBM"", ""Azure""]","[""Infrastructure"", ""IaaS"", … ""Access""]",[],"""Senior IC""",null,"""en""",null,null,null,2024-03-24,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Tools"", ""Azure"", … ""Compute""]","""Tools Azure Infrastructure Sci…"


6. Date cleaning

7. Join with company info, use number of employees

In [63]:
companies = pl.read_csv("companies_sorted.csv")

In [64]:
companies.filter(pl.col("name")=="ibm")

,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
i64,str,str,f64,str,str,str,str,str,i64,i64
5872184,"""ibm""","""ibm.com""",1911.0,"""information technology and ser…","""10001+""","""new york, new york, united sta…","""united states""","""linkedin.com/company/ibm""",274047,716906
1537496,"""ibm""",null,null,"""information technology and ser…","""501 - 1000""",null,null,"""linkedin.com/company/global-va…",280,931
6903843,"""ibm""",null,1912.0,"""information technology and ser…","""11 - 50""",null,null,"""linkedin.com/company/ibm-unite…",7,50
2567154,"""ibm""",null,1914.0,"""information technology and ser…","""1 - 10""","""colombes, ile-de-france, franc…","""france""","""linkedin.com/company/ibm-franc…",3,8
939747,"""ibm""","""mcgeemarketingconsultants.com""",null,"""computer software""","""1 - 10""",null,null,"""linkedin.com/company/mindflow""",2,6
5479744,"""ibm""","""ezsource.com""",2003.0,"""computer software""","""1 - 10""","""mevo modi`im, hamerkaz, israel""","""israel""","""linkedin.com/company/ezlegacy""",1,12
5733310,"""ibm""",null,null,null,"""1 - 10""",null,null,"""linkedin.com/company/ibmdaf""",0,1
2519019,"""ibm""","""ibmconsult.com""",2012.0,"""marketing and advertising""","""1 - 10""",null,null,"""linkedin.com/company/ibmconsul…",0,2
6161271,"""ibm""",null,null,null,"""1 - 10""",null,null,"""linkedin.com/company/fiokware""",0,1


In [65]:
companies_num = companies.select(pl.col("name", "current employee estimate"))

In [66]:
companies_unique = companies_num.group_by("name").agg(pl.col("current employee estimate").sum())


In [67]:
companies_unique

name,current employee estimate
str,i64
"""maan softwares inc.""",4
"""justia""",62
"""alio industries""",9
"""saza solutions""",4
"""grupo integra - bilbao""",3
…,…
"""ekerler mutfak sanayi̇ i̇ç ve …",1
"""global eyes""",0
"""stylish eve""",10


In [68]:
companies_unique.filter(pl.col("name")=="ibm")

name,current employee estimate
str,i64
"""ibm""",274340


In [69]:
whole_df = whole_df.with_columns(name = pl.col("company_name").str.to_lowercase())
whole_df

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags,tags,name
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str],str,str
"""backend developer""","""Praha, Czech Republic""","""Unclear""","""unknown""","""IBM""","""unknown""","[""Docker"", ""GraphQL"", … ""Cloud""]","[""Container"", ""Orchestration"", … ""Databases""]","[""Software""]","""Unclear Seniority""","""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Science"", ""TypeScript"", … ""Programming""]","""Science TypeScript Horizontal …","""ibm"""
"""manufacturing engineer""",null,"""Full-Time""","""false""",null,"""unknown""","[""Sigma""]","[""Tools"", ""Serverless""]","[""Manufacturing""]","""Unclear Seniority""","""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Tools"", ""Serverless"", … ""Sigma""]","""Tools Serverless Manufacturing…",null
"""design engineer, motorized pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""","""unknown""","""ABI Attachments""","""Bachelors""",[],[],"[""Design""]","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Design""]","""Design""","""abi attachments"""
"""cybersecurity engineer""",null,"""Full-Time""","""false""",null,"""Bachelors""","[""AWS"", ""Qualys"", ""Splunk""]","[""Compute"", ""Logging"", … ""Security""]","[""Cybersecurity"", ""Security""]","""Unclear Seniority""","""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""PaaS"", ""Infrastructure"", … ""Security""]","""PaaS Infrastructure Analytics …",null
"""your career so choose wisely w…","""Kolkata, India""","""Full-Time""","""false""","""Adeeba e Services""","""unknown""","[""Objective"", ""C"", … ""Git""]","[""Cloud"", ""Native"", … ""Control""]","[""Software""]","""Unclear Seniority""","""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""iOS"", ""Software"", … ""Version""]","""iOS Software Runtime Storage P…","""adeeba e services"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""fire protection engineer (3+ y…","""1455 Frazee Rd, San Diego, CA …","""Full-Time""","""false""","""Coffman Engineers""","""unknown""","[""Facebook"", ""Instagram"", … ""Twitter""]","[""Tracking"", ""Social"", … ""CDN""]","[""Consulting""]","""Unclear Seniority""","""97500.00""","""en""","""San Diego""","""United States""",2024-01-08,2024-04-01,"""2024""",8,"[""97500.00"", ""97500.00""]","[97500.0, 97500.0]",97500.0,97500.0,"[""Shortening"", ""Twitter"", … ""Link""]","""Shortening Twitter CDN Instagr…","""coffman engineers"""
"""mechanical engineering intern""",null,"""Intern""","""unknown""","""Study and Work""","""unknown""",[],[],"[""Mechanical"", ""Industrial"", ""Engineering""]","""Intern""",null,"""en""",null,null,2024-03-12,2024-03-27,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Industrial"", ""Engineering"", ""Mechanical""]","""Industrial Engineering Mechani…","""study and work"""
"""automation engineer (m/f/d)""","""Puurs""","""Unclear""","""unknown""","""Novartis""","""unknown""",[],[],[],"""Unclear Seniority""",null,"""fr""","""Puurs""","""Belgium""",null,2024-03-16,"""2024""",null,"[null, null]","[null, null]",null,null,[],"""""","""novartis"""


In [70]:
joined_df = whole_df.join(companies_unique, on="name", how="left")
joined_df

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,sample_date,len_comp,est,est_float,compensation,comp_dol,all_tags,tags,name,current employee estimate
str,str,cat,cat,str,cat,list[str],list[str],list[str],cat,str,cat,str,str,date,date,str,u32,list[str],list[f64],f64,f64,list[str],str,str,i64
"""backend developer""","""Praha, Czech Republic""","""Unclear""","""unknown""","""IBM""","""unknown""","[""Docker"", ""GraphQL"", … ""Cloud""]","[""Container"", ""Orchestration"", … ""Databases""]","[""Software""]","""Unclear Seniority""","""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Science"", ""TypeScript"", … ""Programming""]","""Science TypeScript Horizontal …","""ibm""",274340
"""manufacturing engineer""",null,"""Full-Time""","""false""",null,"""unknown""","[""Sigma""]","[""Tools"", ""Serverless""]","[""Manufacturing""]","""Unclear Seniority""","""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Tools"", ""Serverless"", … ""Sigma""]","""Tools Serverless Manufacturing…",null,null
"""design engineer, motorized pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""","""unknown""","""ABI Attachments""","""Bachelors""",[],[],"[""Design""]","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""Design""]","""Design""","""abi attachments""",25
"""cybersecurity engineer""",null,"""Full-Time""","""false""",null,"""Bachelors""","[""AWS"", ""Qualys"", ""Splunk""]","[""Compute"", ""Logging"", … ""Security""]","[""Cybersecurity"", ""Security""]","""Unclear Seniority""","""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""PaaS"", ""Infrastructure"", … ""Security""]","""PaaS Infrastructure Analytics …",null,null
"""your career so choose wisely w…","""Kolkata, India""","""Full-Time""","""false""","""Adeeba e Services""","""unknown""","[""Objective"", ""C"", … ""Git""]","[""Cloud"", ""Native"", … ""Control""]","[""Software""]","""Unclear Seniority""","""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""2023""",0,"["""", """"]","[null, null]",null,null,"[""iOS"", ""Software"", … ""Version""]","""iOS Software Runtime Storage P…","""adeeba e services""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""fire protection engineer (3+ y…","""1455 Frazee Rd, San Diego, CA …","""Full-Time""","""false""","""Coffman Engineers""","""unknown""","[""Facebook"", ""Instagram"", … ""Twitter""]","[""Tracking"", ""Social"", … ""CDN""]","[""Consulting""]","""Unclear Seniority""","""97500.00""","""en""","""San Diego""","""United States""",2024-01-08,2024-04-01,"""2024""",8,"[""97500.00"", ""97500.00""]","[97500.0, 97500.0]",97500.0,97500.0,"[""Shortening"", ""Twitter"", … ""Link""]","""Shortening Twitter CDN Instagr…","""coffman engineers""",279
"""mechanical engineering intern""",null,"""Intern""","""unknown""","""Study and Work""","""unknown""",[],[],"[""Mechanical"", ""Industrial"", ""Engineering""]","""Intern""",null,"""en""",null,null,2024-03-12,2024-03-27,"""2024""",null,"[null, null]","[null, null]",null,null,"[""Industrial"", ""Engineering"", ""Mechanical""]","""Industrial Engineering Mechani…","""study and work""",null
"""automation engineer (m/f/d)""","""Puurs""","""Unclear""","""unknown""","""Novartis""","""unknown""",[],[],[],"""Unclear Seniority""",null,"""fr""","""Puurs""","""Belgium""",null,2024-03-16,"""2024""",null,"[null, null]","[null, null]",null,null,[],"""""","""novartis""",44055


In [71]:
clean_df = joined_df.select(pl.col("job_name", "hours", "remote", "name", "current employee estimate", "education",  "seniority", "city", "country","job_published_at", "sample_date", "tags", "comp_dol"))

In [72]:
clean_df

job_name,hours,remote,name,current employee estimate,education,seniority,city,country,job_published_at,sample_date,tags,comp_dol
str,cat,cat,str,i64,cat,cat,str,str,date,str,str,f64
"""backend developer""","""Unclear""","""unknown""","""ibm""",274340,"""unknown""","""Unclear Seniority""","""Praha""","""Czech Republic""",2023-03-13,"""2023""","""Science TypeScript Horizontal …",null
"""manufacturing engineer""","""Full-Time""","""false""",null,null,"""unknown""","""Unclear Seniority""","""Sterling Heights""","""United States""",2021-10-09,"""2023""","""Tools Serverless Manufacturing…",null
"""design engineer, motorized pro…","""Full-Time""","""unknown""","""abi attachments""",25,"""Bachelors""","""Senior IC""","""Mishawaka""","""United States""",2023-04-28,"""2023""","""Design""",null
"""cybersecurity engineer""","""Full-Time""","""false""",null,null,"""Bachelors""","""Unclear Seniority""","""Herndon""","""United States""",2023-04-03,"""2023""","""PaaS Infrastructure Analytics …",null
"""your career so choose wisely w…","""Full-Time""","""false""","""adeeba e services""",null,"""unknown""","""Unclear Seniority""","""Kolkata""","""India""",2017-01-17,"""2023""","""iOS Software Runtime Storage P…",null
…,…,…,…,…,…,…,…,…,…,…,…,…
"""fire protection engineer (3+ y…","""Full-Time""","""false""","""coffman engineers""",279,"""unknown""","""Unclear Seniority""","""San Diego""","""United States""",2024-01-08,"""2024""","""Shortening Twitter CDN Instagr…",97500.0
"""mechanical engineering intern""","""Intern""","""unknown""","""study and work""",null,"""unknown""","""Intern""",null,null,2024-03-12,"""2024""","""Industrial Engineering Mechani…",null
"""automation engineer (m/f/d)""","""Unclear""","""unknown""","""novartis""",44055,"""unknown""","""Unclear Seniority""","""Puurs""","""Belgium""",null,"""2024""","""""",null


In [73]:
clean_df.write_csv("/home/anopsy/Portfolio/junior_roles/clean_df.csv", separator=",")